In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.{DataFrame,Column,Dataset}
import org.json4s._
import org.json4s.jackson.JsonMethods._

In [2]:
val spark = builder.master("yarn").
config("spark.executor.memory","4G").
config("spark.driver.memory","4G").
config("spark.cores.max","10").
appName("Data Preprocessing").getOrCreate;

spark = org.apache.spark.sql.SparkSession@718f852b


Spark application_1552564637687_17772: Some(http://ip-172-31-38-146.ec2.internal:4042)

In [3]:
val df = spark.read.format("csv").
option("inferSchema", "true").
option("header","true").
option("quote","\"").
option("delimiter",";").
load("train2.csv")

df = [_c0: int, id: int ... 22 more fields]


[_c0: int, id: int ... 22 more fields]

In [4]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- revenue: string (nullable = t

In [5]:
def getNameCollection = udf((entry: String) => {
	// ubah None jadi null
 	val clearNone = entry.replaceAll("None", "null")

 	// jika value json stringnya ' maka harus digantikan dengan "
 	val cleanq = clearNone.replaceAll("'", "\"")


 	// ambil value
 	try {
 		val parser = parse(cleanq)
 		 def scalaFiles = for {
         JObject(child) <- parser
         JField("name", JString(name))  <- child
       } yield name
    	scalaFiles
 	} catch {
 		 case e: Exception => List(null)
 	}

     })

def getList( data: DataFrame, cols: String, col_temp: String) : Array[String] = {
      val list_genres_columns = (
        data.select(cols)
        .withColumn(col_temp,explode(col(cols)))
        .select(col_temp).filter($"$col_temp".isNotNull)
        .distinct.rdd.collect.map(_.getString(0))
    )
    return list_genres_columns
   }

getNameCollection: org.apache.spark.sql.expressions.UserDefinedFunction
getList: (data: org.apache.spark.sql.DataFrame, cols: String, col_temp: String)Array[String]


In [7]:

var df_temp =  df.
withColumn("collection",
		when(
              col("belongs_to_collection").isNotNull,
              getNameCollection(new Column("belongs_to_collection"))
            )
	).
withColumn("newgenres",
		when(
              col("genres").isNotNull,
              getNameCollection(new Column("genres"))
            )
	).
withColumn("newspoken",
		when(
              col("spoken_languages").isNotNull,
              getNameCollection(new Column("spoken_languages"))
            )
	).
withColumn("newkeywords",
		when(
              col("Keywords").isNotNull,
              getNameCollection(new Column("Keywords"))
            )
	).
withColumn("newcast",
		when(
              col("cast").isNotNull,
              getNameCollection(new Column("cast"))
            )
	).
withColumn("newcrew",
		when(
              col("crew").isNotNull,
              getNameCollection(new Column("crew"))
            )
	).
withColumn("newproduction_countries",
		when(
              col("production_countries").isNotNull,
              getNameCollection(new Column("production_countries"))
            )
	).
withColumn("newproduction_companies",
		when(
              col("production_companies").isNotNull,
              getNameCollection(new Column("production_companies"))
            )
)             

df_temp = [_c0: int, id: int ... 30 more fields]


[_c0: int, id: int ... 30 more fields]

In [8]:

//df_temp.select("newproduction_companies").filter($"newproduction_companies".isNotNull).show()
// convert column to list and filtered null value
var list_genres_columns = getList(df_temp,"newgenres","genres")
var list_spoken_columns = getList(df_temp,"newspoken","spoken")
var list_keywords_columns = getList(df_temp,"newkeywords","keywords")
var list_cast_columns = getList(df_temp,"newcast","cast")
var list_crew_columns = getList(df_temp,"newcrew","crew")
var list_country_columns = getList(df_temp,"newproduction_countries","prod_countries")
var list_company_columns = getList(df_temp,"newproduction_companies","prod_s")

list_genres_columns = Array(Crime, Romance, TV Movie, Thriller, Adventure, Foreign, Drama, War, Documentary, Family, Fantasy, History, Mystery, Animation, Music, Science Fiction, Horror, Western, Comedy, Action)
list_spoken_columns = Array(Wolof, 한국어/조선말, Hrvatski, Bahasa indonesia, Gaeilge, Deutsch, Latin, Pусский, Français, Norsk, Somali, پښتو, Kiswahili, हिन्दी, Polski, Română, العربية, বাংলা, isiZulu, Magyar, עִבְרִית, Slovenčina, English, suomi, Tiếng Việt, Español, қазақ, ਪੰਜਾਬੀ, Afrikaans, తెలుగు, Dansk, ภาษาไทย, Český, 广州话 / 廣州話, Eesti, Português, Srpski, Català, Nederlands, Türkçe, فارسی, Український, اردو, 日本語, български език, தமிழ், "", Esperanto, shqip, Italiano, euskera, 普通话, ελληνικά, svenska, No Language, Íslenska)


list_keywords_columns: Arra...


[Wolof, 한국어/조선말, Hrvatski, Bahasa indonesia, Gaeilge, Deutsch, Latin, Pусский, Français, Norsk, Somali, پښتو, Kiswahili, हिन्दी, Polski, Română, العربية, বাংলা, isiZulu, Magyar, עִבְרִית, Slovenčina, English, suomi, Tiếng Việt, Español, қазақ, ਪੰਜਾਬੀ, Afrikaans, తెలుగు, Dansk, ภาษาไทย, Český, 广州话 / 廣州話, Eesti, Português, Srpski, Català, Nederlands, Türkçe, فارسی, Український, اردو, 日本語, български език, தமிழ், , Esperanto, shqip, Italiano, euskera, 普通话, ελληνικά, svenska, No Language, Íslenska]

In [ ]:
// create genres columns from newgenres value
for ( genre <- list_genres_columns ){
     df_temp = df_temp.
    withColumn("genre_"+genre.split(" ").mkString("_"),when(array_contains(col("newgenres"),genre),1).
              otherwise(0))
}
for ( spoken <- list_spoken_columns ){
     df_temp = df_temp.
    withColumn("spoken_"+spoken.split(" ").mkString("_"),when(array_contains(col("newspoken"),spoken),1).
              otherwise(0))
}
for ( keywords <- list_keywords_columns ){
     df_temp = df_temp.
    withColumn("keyword_"+keywords.split(" ").mkString("_"),when(array_contains(col("newkeywords"),keywords),1).
              otherwise(0))
}
for ( cast <- list_cast_columns ){
     df_temp = df_temp.
    withColumn("cast_"+cast.split(" ").mkString("_"),when(array_contains(col("newcast"),cast),1).
              otherwise(0))
}
for ( crew <- list_crew_columns ){
     df_temp = df_temp.
    withColumn("crew_"+crew.split(" ").mkString("_"),when(array_contains(col("newcrew"),crew),1).
              otherwise(0))
}
for ( country <- list_country_columns ){
     df_temp = df_temp.
    withColumn("country_"+country.split(" ").mkString("_"),when(array_contains(col("newproduction_countries"),country),1).
              otherwise(0))
}
for ( company <- list_company_columns ){
     df_temp = df_temp.
    withColumn("company_"+company.split(" ").mkString("_"),when(array_contains(col("newproduction_companies"),company),1).
              otherwise(0))
}

Exception in thread "dispatcher-event-loop-3" Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: GC overhead limit exceeded
Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: GC overhead limit exceeded
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at scala.runtime.ObjectRef.create(ObjectRef.java:22)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:88)
	at org.apache.spark.rpc.netty.Dispatcher$MessageLoop.run(Dispatcher.scala:221)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:617)
	at java.lang.Thread.run(Thread.java:745)


Name: java.lang.OutOfMemoryError
Message: GC overhead limit exceeded
StackTrace: 